In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/gdnhealthcare.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,585876545266286592,Wed Apr 08 18:47:08 +0000 2015,What is palliative care like in India? One GP ...
1,585861945535791106,Wed Apr 08 17:49:07 +0000 2015,Most viewed this week: I loved being a midwife...
2,585859917350731777,Wed Apr 08 17:41:03 +0000 2015,How can technology improve mental health waiti...
3,585844465199407104,Wed Apr 08 16:39:39 +0000 2015,In case you missed it: Why the #NHS shouldn’t ...
4,585800529739001856,Wed Apr 08 13:45:04 +0000 2015,India is a stark example of how commodifying h...


In [3]:
frame.shape

(2993, 3)

In [4]:
frame.loc[:,'title'][0]

'What is palliative care like in India? One GP trainee explains all: http://gu.com/p/47a4g/stw'

In [5]:
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(lambda row: split_txt_form_url(regex_url, row))
frame.head()

http://gu.com/p/47a4g/stw
http://gu.com/p/46zc4/stw
http://gu.com/p/476ky/stw
http://gu.com/p/477tc/stw
http://gu.com/p/47a4g/stw
http://www.theguardian.com/healthcare-network/2015/apr/07/why-nhs-should-not-hog-limelight-general-election
http://gu.com/p/47a4g/stw
http://gu.com/p/47ajt/stw
http://www.theguardian.com/healthcare-network/2015/apr/02/technology-mental-health-waiting-times
http://gu.com/p/477tc/stw
http://gu.com/p/47a4g/stw
http://gu.com/p/477tc/stw
http://www.vice.com/en
http://gu.com/p/476fa/stw
http://gu.com/p/47a5x/stw
http://gu.com/p/477tc/stw
http://www.theguardian.com/healthcare-network/2015/apr/07/why-nhs-should-not-hog-limelight-general-election
http://gu.com/p/478kc/stw
http://gu.com/p/477tc/stw
http://gu.com/p/477tc
http://gu.com/p/476ky/stw
http://gu.com/p/45445/stw
http://sumall.com/myweek
http://gu.com/p/46hnm/stw
http://gu.com/p/472d5/stw
http://gu.com/p/46gy5/stw
http://gu.com/p/476ky/stw
http://gu.com/p/45yzh/stw
http://gu.com/p/475ch/stw
http://gu.com/p/474

http://gu.com/p/435qh/tw
http://gu.com/p/434ja/tw
http://gu.com/p/435tf/tw
http://gu.com/p/4vkmy/tw
http://gu.com/p/436mk/tw
http://gu.com/p/42ydx/tw
http://gu.com/p/434ja/tw
http://gu.com/p/43vej/tw
http://gu.com/p/42qen/tw
http://gu.com/p/42qax/tw
http://gu.com/p/43vac/tw
http://gu.com/p/434nk/tw
http://gu.com/p/435qh/tw
http://gu.com/p/435tf/tw
http://gu.com/p/435qh/tw
http://gu.com/p/437g5/tw
http://gu.com/p/43vej/tw
http://gu.com/p/434ja/tw
http://gu.com/p/43vej/tw
http://gu.com/p/434ja/tw
http://gu.com/p/43vej/tw
http://gu.com/p/434ja/tw
http://sumall.com/myweek
http://gu.com/p/435tf/tw
http://gu.com/p/434nk/tw
http://gu.com/p/4vkmy/tw
http://gu.com/p/435tf/tw
http://gu.com/p/435tf/tw
http://gu.com/p/434nk/tw
http://gu.com/p/436mk/tw
http://www.theguardian.com/healthcare-network/2014/nov/05/nhs-listen-act-patient-complaints
http://www.theguardian.com/healthcare-network/2014/nov/05/nhs-listen-act-patient-complaints
http://www.theguardian.com/healthcare-network/2014/nov/04/big-data

http://goo.gl/8F1hlC
http://bit.ly/1xY24fE
http://gu.com/p/44m6p/stw
http://gu.com/p/44m8z/tw
http://bit.ly/1xY24fE
http://gu.com/p/44y9k/stw
http://bit.ly/1AvMxBE
http://bit.ly/1xwJWaE
http://gu.com/p/44m4e/tw
http://bit.ly/1tNqMsa
http://bit.ly/1BKYK4G
http://bit.ly/1HxWMIX
http://bit.ly/1Azn1N6
http://bit.ly/1tNqMsa
http://bit.ly/1BKYK4G
http://www.theguardian.com/healthcare-network/2015/jan/08/nhs-trusts-tough-choices-future
http://ow.ly/GZXZP
http://gu.com/p/44y7d/tw
http://bit.ly/1tNqMsa
http://gu.com/p/44yag/stw
http://bit.ly/1BKYK4G
http://gu.com/p/44k7h/tw
http://gu.com/p/44yvb/tw
http://gu.com/p/44kkj/stw
http://bit.ly/14ueOhi
http://bit.ly/1HxWMIX
http://bit.ly/1BpyIpV
http://bit.ly/14ueOhi
http://bit.ly/1Azn1N6
http://gu.com/p/44k8t/stw
http://bit.ly/14ueOhi
http://bit.ly/1HxWMIX
http://gu.com/p/44hgq/stw
http://t.
http://gu.com/p/44k8t/stw
http://gu.com/p/44ka9/tw
http://bit.ly/14ueOhi
http://www.theguardian.com/healthcare-network/2015/jan/06/cracks-began-show-nhs-kailash-

,tweetid,date,title,url
0,585876545266286592,Wed Apr 08 18:47:08 +0000 2015,What is palliative care like in India? One GP ...,http://gu.com/p/47a4g/stw
1,585861945535791106,Wed Apr 08 17:49:07 +0000 2015,Most viewed this week: I loved being a midwife...,http://gu.com/p/46zc4/stw
2,585859917350731777,Wed Apr 08 17:41:03 +0000 2015,How can technology improve mental health waiti...,http://gu.com/p/476ky/stw
3,585844465199407104,Wed Apr 08 16:39:39 +0000 2015,In case you missed it: Why the #NHS shouldn’t ...,http://gu.com/p/477tc/stw
4,585800529739001856,Wed Apr 08 13:45:04 +0000 2015,India is a stark example of how commodifying h...,http://gu.com/p/47a4g/stw


In [6]:
frame_urls = frame['url'].values

In [7]:
# Run for this jupyter notebook
title_attempt = True
for key,url in enumerate(frame_urls):

    #     url = 'http://bit.ly/VymaIc'
    try:
        # Start the connection
        print(url)
        if len(url):
            if "http://" not in url:
                url = "http://" + url
            res = requests.get(url)

            # Check status header
            if res.status_code == 200:
                print("Header: ", 200)

                # Initialize BeautifulSoup()
                soup = BeautifulSoup(res.content, 'lxml')

                # Get return content as text
                # text_html = res.text

                # Add columns to dataframe
                if title_attempt:
                    df_temp = pd.DataFrame(columns=['author','content','tags'])
                    frame = pd.concat([frame,df_temp])
                    title_attempt = False

                ahref = soup.find('a',{'class':'article_link'})
                if ahref:
                    author = ahref.text            
        #             frame['author'][key] = author[:author.index(",")].lstrip()
                    frame['author'][key] = author.lstrip()
                else:
                    frame['author'] = ''
                frame['content'][key] = soup.find('div',{'class':'article-body'})
            else:
                print("[ERROR]: ", res.status_code)
        else:
            frame['author'][key] = ''
            frame['content'] = ''
        frame['tags'][key] = get_hastags_out(regex_hastags,frame['title'][key])
        frame['title'][key] = stript_hastags(frame['title'][key])
    except (RuntimeError, TypeError, NameError):
        pass

http://gu.com/p/47a4g/stw
Header:  200


scripts.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Get hastags out of string.
scripts.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return sample.replace("#",'')
scripts.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result = dict()
scripts.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

http://gu.com/p/46zc4/stw
Header:  200
http://gu.com/p/476ky/stw
Header:  200
http://gu.com/p/477tc/stw
Header:  200
http://gu.com/p/47a4g/stw
Header:  200



scripts.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Scrapt content base on xpath command.


http://www.theguardian.com/healthcare-network/2015/apr/07/why-nhs-should-not-hog-limelight-general-election
Header:  200

http://gu.com/p/47a4g/stw
Header:  200
http://gu.com/p/47ajt/stw
Header:  200
http://www.theguardian.com/healthcare-network/2015/apr/02/technology-mental-health-waiting-times
Header:  200
http://gu.com/p/477tc/stw
Header:  200
http://gu.com/p/47a4g/stw
Header:  200
http://gu.com/p/477tc/stw
Header:  200
http://www.vice.com/en
[ERROR]:  503
http://gu.com/p/476fa/stw
Header:  200


http://gu.com/p/47a5x/stw
Header:  200
http://gu.com/p/477tc/stw
Header:  200

http://www.theguardian.com/healthcare-network/2015/apr/07/why-nhs-should-not-hog-limelight-general-election
Header:  200
http://gu.com/p/478kc/stw
Header:  200
http://gu.com/p/477tc/stw
Header:  200

http://gu.com/p/477tc
Header:  200

http://gu.com/p/476ky/stw
Header:  200

http://gu.com/p/45445/stw
Header:  200
http://sumall.com/myweek
Header:  200
http://gu.com/p/46hnm/stw
Header:  200
http://gu.com/p/472d5/st

Header:  200
http://bit.ly/1xrFD2P
Header:  200

http://gu.com/p/46peh/stw
Header:  200
http://t.co/fXQeR
[ERROR]:  404

http://gu.com/p/46zc4/stw
Header:  200
http://gu.com/p/46zc4/stw
Header:  200
http://gu.com/p/45nt6/stw
Header:  200
http://gu.com/p/46ya4/stw
Header:  200
http://gu.com/p/46k6c/stw
Header:  200
http://gu.com/p/46tf5/stw
Header:  200
http://gu.com/p/46thp/stw
Header:  200
http://gu.com/p/46kgq/stw
Header:  200
http://gu.com/p/46m98/stw
Header:  200
http://gu.com/p/46kgq/stw
Header:  200
http://gu.com/p/46n7z/stw
Header:  200
http://gu.com/p/46hqp/stw
Header:  200
http://gu.com/p/46nzv/stw
Header:  200
http://gu.com/p/46kgq/stw
Header:  200
http://gu.com/p/46ya4/stw
Header:  200
http://gu.com/p/46gy5/stw
Header:  200
http://gu.com/p/46thp/stw
Header:  200
http://gu.com/p/46kgq/stw
Header:  200
http://gu.com/p/46k6c/stw
Header:  200
http://gu.com/p/46v6j/stw
Header:  200
http://gu.com/p/46ya4/stw
Header:  200
http://gu.com/p/46kgq/stw
Header:  200
http://gu.com/p/46m98

ConnectionError: HTTPConnectionPool(host='https', port=80): Max retries exceeded with url: //t.co/scRR (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000027758797630>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [ ]:
frame.head()

In [ ]:
frame.shape